In [1]:
import json
import time
import os
from google.colab import drive, files


In [2]:
# Montar Google Drive en Colab
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def load_json_file(file_path):
    """Carga un archivo JSON y maneja errores."""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: El archivo {file_path} no fue encontrado.")
        return None
    except json.JSONDecodeError:
        print(f"Error: El archivo {file_path} no tiene un formato JSON válido.")
        return None

def compute_total_cost(price_catalogue, sales_record):
    """Calcula el costo total de las ventas y maneja errores."""
    total_cost = 0.0
    errors = []

    for sale in sales_record:
        product_name = sale.get("Product")
        quantity = sale.get("Quantity")

        if not product_name or not isinstance(quantity, (int, float)) or quantity <= 0:
            errors.append(f"Advertencia: Venta inválida encontrada: {sale}")
            continue

        product_price = next((item["price"] for item in price_catalogue if item["title"] == product_name), None)

        if product_price is None:
            errors.append(f"Advertencia: Producto '{product_name}' no encontrado en el catálogo.")
            continue

        total_cost += float(product_price) * int(quantity)

    return total_cost, errors

def process_sales_files(sales_files, price_catalogue):
    """Procesa los archivos de ventas y calcula el total."""
    results = {}

    for sales_file in sales_files:
        sales_data = load_json_file(sales_file)

        if sales_data is None:
            print(f"Error: No se pudo cargar {sales_file}.")
            continue

        test_case = os.path.basename(sales_file).split(".")[0]
        total_cost, errors = compute_total_cost(price_catalogue, sales_data)
        results[test_case] = total_cost

        print(f"Total Sales for {test_case}: ${total_cost:.2f}")
        if errors:
            print(f"Errors in {test_case}:")
            for error in errors:
                print(error)

    return results

def save_results(results, elapsed_time):
    """Guarda los resultados en un archivo de texto y lo descarga."""
    output_file = "/content/drive/My Drive/Colab Notebooks/Results.txt"
    with open(output_file, "w", encoding="utf-8") as file:
        file.write("TOTAL\n")
        for test_case, total in results.items():
            file.write(f"{test_case}\t{total:.2f}\n")
        file.write(f"\nExecution Time: {elapsed_time:.2f} seconds\n")

    print(f"Results saved in {output_file}")
    files.download(output_file)

def main():
    """Función principal para ejecutar el procesamiento."""
    start_time = time.time()

    base_folder = "/content/drive/My Drive/Colab Notebooks"
    price_catalogue_file = os.path.join(base_folder, "TC1.ProductList.json")
    sales_files = [
        os.path.join(base_folder, "TC1.Sales.json"),
        os.path.join(base_folder, "TC2.Sales.json"),
        os.path.join(base_folder, "TC3.Sales.json"),
    ]

    # Cargar catálogo de productos
    price_catalogue = load_json_file(price_catalogue_file)
    if price_catalogue is None:
        print("Error: No se pudo cargar el catálogo de productos.")
        return

    # Procesar archivos de ventas
    results = process_sales_files(sales_files, price_catalogue)

    # Guardar resultados
    elapsed_time = time.time() - start_time
    save_results(results, elapsed_time)

if __name__ == "__main__":
    main()

Total Sales for TC1: $2481.86
Total Sales for TC2: $169478.22
Errors in TC2:
Advertencia: Venta inválida encontrada: {'SALE_ID': 8, 'SALE_Date': '02/12/23', 'Product': 'Fresh blueberries', 'Quantity': -35}
Advertencia: Venta inválida encontrada: {'SALE_ID': 10, 'SALE_Date': '02/12/23', 'Product': 'Green smoothie', 'Quantity': -123}
Total Sales for TC3: $168145.36
Errors in TC3:
Advertencia: Producto 'Elotes' no encontrado en el catálogo.
Advertencia: Venta inválida encontrada: {'SALE_ID': 8, 'SALE_Date': '02/12/23', 'Product': 'Fresh blueberries', 'Quantity': -35}
Advertencia: Producto 'Frijoles' no encontrado en el catálogo.
Advertencia: Venta inválida encontrada: {'SALE_ID': 10, 'SALE_Date': '02/12/23', 'Product': 'Green smoothie', 'Quantity': -123}
Results saved in /content/drive/My Drive/Colab Notebooks/Results.txt


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>